<pre>
 ____   ____    _    ____  _____                          _      _     
|  _ \ / ___|  / \  / ___|| ____|     _ __ ___   ___   __| | ___| |___ 
| | | | |     / _ \ \___ \|  _| _____| '_ ` _ \ / _ \ / _` |/ _ \ / __|
| |_| | |___ / ___ \ ___) | |__|_____| | | | | | (_) | (_| |  __/ \__ \
|____/ \____/_/   \_\____/|_____|    |_| |_| |_|\___/ \__,_|\___|_|___/
                                                                        
</pre>

# DCASE-models Notebooks
Python Notebooks for [DCASE-models](https://github.com/pzinemanas/DCASE-models)

---

### About 
This Notebook reproduces *some* results for **Enviromental Sound Classification** presented in:
<ul>
<li><a href="http://www.justinsalamon.com/uploads/4/3/9/4/4394963/salamon_cnn-aug-env_ieeespl_2017.pdf"><strong>
    Deep Convolutional Neural Networks and Data Augmentation For Environmental Sound Classification</strong></a>
    J. Salamon and J. P. Bello IEEE Signal Processing Letters, 24(3), pages 279 - 283, 2017.
    <br>
   <a type="button" class="btn btn-default btn-xs" target="_blank" href="http://www.justinsalamon.com/uploads/4/3/9/4/4394963/salamon_cnn-aug-env_ieeespl_2017.pdf"> PDF </a>
   <a type="button" class="btn btn-default btn-xs" target="_blank" href="https://ieeexplore.ieee.org/document/7829341"> IEEE</a>
    </li>   
</ul>

### Overview

The paper introduces an adaptation of the Convolutional Neural Network (CNN) for environmental
sound classification, using the [UrbanSound8k dataset](https://urbansounddataset.weebly.com/urbansound8k.html).



### Organization

The Notebook is organized into the following sections.
* [1. Load parameters](#LoadParameters)
* [2. Initialize data generator](#Init)
* [3. Extract Features](#ExtractFeatures)
* [4. Load data](#loadData)
* [5. Fit Scaler](#scaler)
* [6. Initialize model](#initmod)
* [7. Train model](#train)
* [8. Evaluate model](#eval)

In [ ]:
%load_ext autoreload
%autoreload 2
rootdir_path = '../../'
import sys
import os
import json
import warnings
import glob
import numpy as np
import argparse

sys.path.append(rootdir_path)
from dcase_models.utils.files import load_json, mkdir_if_not_exists
from dcase_models.data.data_generator import DataGenerator
from dcase_models.data.datasets import UrbanSound8k
from dcase_models.data.datasets import ESC10
from dcase_models.data.features import MelSpectrogram
from dcase_models.model.models import SB_CNN
from dcase_models.data.scaler import Scaler
from dcase_models.utils.files import load_json



os.environ["CUDA_VISIBLE_DEVICES"] = "1"

<a id="LoadeParameters"></a>
## 1. Load parameters

Dataset, feature extraction and training parameters are stored in a json file on the root directory.

In [ ]:
# load all parameters from json file
params = load_json(os.path.join(rootdir_path, 'parameters.json'))
# set the dataset we are going to use
dataset = 'UrbanSound8k'

# get dataset parameters
params_dataset = params["datasets"][dataset]

# get feature extraction parameters
params_features = params["features"]

# get training parameters
params_train = params["train"]

Check that the values of the parameters are correct.

In [ ]:
# print the dataset parameters 
print("Dataset Parameters:\n", json.dumps(params_dataset, indent=4, sort_keys=True))
# print feature extraction parameters 
print("Features' Parameters:\n",json.dumps(params_features, indent=4, sort_keys=True))
# print training parameters 
print("Training Parameters:\n",json.dumps(params_train, indent=4, sort_keys=True))

<a id="init"></a>
## 2. Initialize Feature Extractor and Data Generator

In [ ]:
# Initialize Feature Extractor
feature_extractor = MelSpectrogram(sequence_time=params_features['sequence_time'], \
                                            sequence_hop_time=params_features['sequence_hop_time'], 
                                            audio_win=params_features['audio_win'], 
                                            audio_hop=params_features['audio_hop'], 
                                            n_fft=params_features['Spectrogram']['n_fft'], 
                                            sr=params_features['sr'])


# Init Data Generator
dataset = UrbanSound8k(os.path.join(rootdir_path, params_dataset["dataset_path"]))

Check if dataset exists, and download it if doesn't.

In [ ]:
dataset.download_dataset()

In [ ]:
data_generator = DataGenerator(dataset, feature_extractor)

<a id="ExtractFeatures"></a>
## 3. Extract Features

In [ ]:
data_generator.extract_features()
print('Done!')

<a id="load_data"></a>
## 4. Load data

In [ ]:
print('Loading data... ')
data_generator.load_data()
print('Done!')

<a id="scaler"></a>
## 5. Fit scaler

In [ ]:
fold_test = 'fold1'

X_train, Y_train, X_val, Y_val = data_generator.get_data_for_training(fold_test)
scaler = Scaler(normalizer=normalizer)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

<a id="initmodel"></a>
## 6. Init Model

In [ ]:
n_frames_cnn = X_train.shape[1]
n_freq_cnn = X_train.shape[2]
n_classes = Y_train.shape[1]
print(n_frames_cnn, n_freq_cnn, n_classes)

model_container = SB_CNN(model=None, model_path=None, n_classes=n_classes, 
                         n_frames_cnn=n_frames_cnn, n_freq_cnn=n_freq_cnn)

model_container.model.summary()

<a id="train"></a>
## 7. Train model

In [ ]:
exp_folder = './'


model_container.train(X_train, Y_train, X_val, Y_val, weights_path=exp_folder, **params_train)

<a id="Eval"></a>
## 8. Evaluate Model

In [ ]:
# Load best_weights
model_container.load_model_weights(exp_folder)

# Test model
X_test, Y_test = data_generator.get_data_for_testing(fold_test)
X_test = scaler.transform(X_test)
results = model_container.evaluate(X_test, Y_test)

print(results['accuracy'])